In [1]:
!pip install redis

In [2]:
import redis
r = redis.Redis(
	host='redis',
	port=6379,
	db=0,
	decode_responses=True
)

In [3]:
#Esta forma es la más común
import csv
from datetime import datetime
with open("clean_file.csv", "r") as f:
    lector = csv.DictReader(f)
    for fila in lector:
        #Recorremos la filas en forma de diccionario
        fila["score"] = float(fila["score"])
        fecha_limpia = fila["date_x"].strip()  # quita espacios
        fila["date_x"] = datetime.strptime(fecha_limpia, "%m/%d/%Y")
        r.rpush("peliculas:orden",fila["names"])
        r.hset(f"{fila['names']}", mapping={
            "fecha": str(fila["date_x"]),
            "score": fila["score"],
            "genero": fila["genre"],
            "overview": fila["overview"],
            "crew": fila["crew"],
            "titulo_Original" : fila["orig_title"],
            "status": fila["status"],
            "Idioma Original": fila["orig_lang"],
            "budget_x": fila["budget_x"],
            "revenue":fila["revenue"],
            "country":fila["country"]            
        })
        r.zadd("ranking:score",{fila["names"]:fila["score"]})
        r.zadd("ranking:revenue",{fila["names"]:fila["score"]})
        
       
        

In [4]:
r.zrevrange("ranking:score",0,3)

['The Chosen: Season 3  - Episodes 1 & 2',
 'Simulant',
 'Pretty Young Sister 4',
 'Porno document: Toruko tokkyû bin']

In [5]:
for nombre in r.zrange("ranking:revenue",0,5):
    print(nombre, r.zscore("ranking:revenue",nombre))

77+33=? 0.0
99 Moons 0.0
A Newly Wedded Couple's Sex Life 0.0
ACA 155: Karginov vs. Silva 0.0
ARES 14 0.0
About Us But Not About Us 0.0


In [6]:
r.lrange("peliculas:orden",0,3)

['Creed III',
 'Avatar: The Way of Water',
 'The Super Mario Bros. Movie',
 'Mummies']

In [7]:
print("obtener el revenue del qe tiene mayor score")
r.zrevrange("ranking:revenue",0,0)
r.hget("The Chosen: Season 3  - Episodes 1 & 2","revenue")

obtener el revenue del qe tiene mayor score


'1569323843.8'

In [8]:
r.zrevrange("ranking:score",0,3)


['The Chosen: Season 3  - Episodes 1 & 2',
 'Simulant',
 'Pretty Young Sister 4',
 'Porno document: Toruko tokkyû bin']

In [9]:
r.hget("The Chosen: Season 3  - Episodes 1 & 2","Idioma Original")

' English'

In [10]:
r.hget("Porno document: Toruko tokkyû bin","Idioma Original")

' Japanese'

In [11]:
r.hget("Pretty Young Sister 4","Idioma Original")

' Korean'

In [121]:
r.flushdb()

True